# Output
This notebook generates the paragraph about the microCT-scanning from logfiles of the scans.
And an XLS sheet with the details from the scans.

In [ ]:
import platform
import os
import pandas
import glob
import numpy

In [ ]:
# Import our own parsing functions which we've added as submodule
from BrukerSkyScanLogfileRuminator.parsing_functions import *

We select a folder and go through *each* subfolder there...

In [ ]:
# Load the log files from the archive
if 'Linux' in platform.system():
    BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-uct', 'Archiv_Tape')
else:
    BasePath = os.path.join('R:\\', 'Archiv_Tape')
# Select relevant folder
Root = os.path.join(BasePath, '*Aaldijk*')
print('We are loading all the data from %s' % Root)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Look for *all* log files in the selected folder
Data['LogFile'] = glob.glob(os.path.join(Root, '**', '*.log'), recursive=True)

In [ ]:
Data['LogFile'][0]

In [ ]:
# Save us a less verbose relative path for the log files
Data['LogFileRelativePath'] = [l[len(os.path.split(Root)[0])+1:] for l in Data['LogFile']]

In [ ]:
Data['LogFileRelativePath'][0]

In [ ]:
# Use only the 'proj' log files
for c, row in Data.iterrows():
    if 'rec' not in row.LogFile:
        Data.drop([c], inplace=True)
    if 'rectmp' in row.LogFile:
        Data.drop([c], inplace=True)
    if 'ctan.log' in row.LogFile:
        # Remove log file from CTAn
        Data.drop([c], inplace=True)

In [ ]:
# for l in Data.LogFile:
#     print(l)

In [ ]:
# For the manuscript, we use only the 'Foetus02' and 'Mouse01' scans, so let's only use these logs:

In [ ]:
# Use only the 'proj' log files
for c, row in Data.iterrows():
    if ('Foetus02' not in row.LogFile) & ('Mouse01' not in row.LogFile):  # Exclude all other scans
        Data.drop([c], inplace=True)
    elif 'Registration' in row.LogFile:  # We've tried to register scans with DataViewer, which we also want to exclude
        Data.drop([c], inplace=True)        

In [ ]:
Data

In [ ]:
# Generate descritive 'sample' name
Data['Sample'] = [os.path.basename(os.path.dirname(os.path.dirname(l))).split('_')[0] for l in Data['LogFile']]
# Sort 

In [ ]:
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Software'] = [controlsoftware(log) for log in Data['LogFile']]

In [ ]:
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Voxelsize_rounded'] = [round(vs,1) for vs in Data['Voxelsize']]

In [ ]:
Data['Source'] = [source(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]

In [ ]:
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]

In [ ]:
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['NumberOfProjections'] = [numproj(log) for log in Data['LogFile']]
Data['CameraSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Wide'] = [overlapscan(log) for log in Data.LogFile]
Data['ThreeSixtyScan'] = [threesixtyscan(log) for log in Data['LogFile']]

In [ ]:
Data['RingRemoval'] = [ringremoval(log) for log in Data['LogFile']]
Data['Beamhardening'] = [beamhardening(log) for log in Data['LogFile']]
Data['GrayValueMax'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]

In [ ]:
Data['Exposure'] = [exposuretime(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]

In [ ]:
Data['Duration'] = [duration(log) for log in Data['LogFile']]

In [ ]:
Data['Version'] = [nreconversion(log) for log in Data['LogFile']]

In [ ]:
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]
# Calculate time 'spent' since start (for each sample separately)
# Data['Time passed'] = [sd - Data['Scan date'].min() for sd in Data['Scan date']]
for sample in Data.Sample.unique():
    print('Sample %s scanned from %s to %s' % (sample, 
                                               Data[Data.Sample == sample]['Scan date'].min().date(),
                                               Data[Data.Sample == sample]['Scan date'].max().date()))                                            
    for c, row in Data[Data.Sample == sample].iterrows():
        Data.at[c, 'Time passed'] = row['Scan date'] - Data[Data.Sample == sample]['Scan date'].min()
# Also extract days, rounded
Data['Days passed'] = [t.round('d') for t in Data['Time passed']]

In [ ]:
# Sort our dataframe by Sample and time passed
Data.sort_values(['Sample', 'Time passed'], inplace=True)

In [ ]:
for i in Data:
    print("'%s'," % i)

In [ ]:
Data.head()

In [ ]:
# Save to 'data' subfolder here
Data[['LogFileRelativePath',
      'Sample',
      'Scanner',
      'Software',
      'Voxelsize',
      'Voxelsize_rounded',
      'Source',
      'Camera',
      'Voltage',
      'Current',
      'Filter',
      'Stacks',
      'NumberOfProjections',
      'CameraSize',
      'RotationStep',
      'Wide',
      'ThreeSixtyScan',
      'RingRemoval',
      'Beamhardening',
      'GrayValueMax',
      'Exposure',
      'Averaging',
      'Duration',
      'Version',
      'Scan date',
      'Time passed',
      'Days passed']].to_csv(os.path.join('data', 'ScanDetails.Foetus.Mouse.csv'))

In [ ]:
# Save to research storage folder of Dea
Data[['LogFileRelativePath',
      'Sample',
      'Scanner',
      'Software',
      'Voxelsize',
      'Voxelsize_rounded',
      'Source',
      'Camera',
      'Voltage',
      'Current',
      'Filter',
      'Stacks',
      'NumberOfProjections',
      'CameraSize',
      'RotationStep',
      'Wide',
      'ThreeSixtyScan',
      'RingRemoval',
      'Beamhardening',
      'GrayValueMax',
      'Exposure',
      'Averaging',
      'Duration',
      'Version',
      'Scan date',
      'Time passed',
      'Days passed']].to_excel(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Data', 'ScanDetails.Foetus.Mouse.xlsx'))

----

My microct blurb from http://simp.ly/publish/NBhZhH

In [ ]:
print('Based on the %s log files read from %s' % (len(Data), Root))

In [ ]:
print('After $PREPARATION, the',
      len(Data.Sample.unique()),
      'samples were imaged on a Bruker',
      " OR ".join(str(value) for value in Data.Scanner.unique()),
      'high-resolution microtomography machine (Control software version',
      " OR ".join(str(value) for value in Data.Software.unique()) + 
      ', Bruker microCT, Kontich, Belgium).')

In [ ]:
print('The machine is equipped with a',
      " OR ".join(str(value) for value in Data.Source.unique()),
      'X-ray source and a',
      " OR ".join(str(value) for value in Data.Camera.unique()),
      'camera.')

In [ ]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    print('We scanned at %s different time points' % len(Data[Data.Sample == sample]['Days passed'].unique()))
    print('From %s to %s' % (Data[Data.Sample == sample]['Scan date'].min().date(),
                             Data[Data.Sample == sample]['Scan date'].max().date()))
    print('Over a total of %s days' % Data[Data.Sample == sample]['Days passed'].max().days)
    print('I.e. on average one scan every %s days' % round(Data[Data.Sample == sample]['Days passed'].max().days / len(Data[Data.Sample == sample]['Days passed'].unique())))
    print(80 * '-')

In [ ]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    print('The resulting voxel size was %s μm' % Data[Data.Sample == sample].Voxelsize.unique())
    print('The X-ray source was set to a tube voltage of', 
          " OR ".join(str(value) for value in Data[Data.Sample == sample].Voltage.unique()),
          'kV and a tube current of',
          " OR ".join(str(value) for value in Data[Data.Sample == sample].Current.unique()),
          'µA, the x-ray spectrum was', end=' ')
    if Data[Data.Sample == sample].Filter.unique():
        print('filtered by', " OR ".join(str(value) for value in Data[Data.Sample == sample].Filter.unique()), end='.')
    else:
        print('not filtered', end='.')
    print()
    print(80 * '-')

In [ ]:
for sample in Data.Sample.unique():
    print('For the "%s" scans' % sample)
    for ts in Data[Data.Sample == sample].ThreeSixtyScan.unique():
        print('For 360°==%s' % ts)
        print(Data[Data.Sample == sample][Data[Data.Sample == sample].ThreeSixtyScan == ts][['Voxelsize', 'Stacks',
                                                                                             'CameraSize', 'NumberOfProjections', 'RotationStep',
                                                                                             'Exposure', 'Duration']])
        print(80*'-')

In [ ]:
for vs in Data.Voxelsize.unique():
    print(vs)

In [ ]:
for vs in Data.Voxelsize.unique():
    print('For each of the %s %s scans scanned with %s μm, we recorded a set of either' % (len(Data[Data.Voxelsize == vs]),
                                                                                           Data[Data.Voxelsize == vs]['Sample'].unique()[0],
                                                                                           round(vs)),
          end=' ')
    if Data[Data.Voxelsize == vs].Filter.unique().tolist()[0]:   
        print(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].Stacks.unique()),
              'stacked scans overlapping the sample height, each stack was recorded with', end=' ')
    print(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].NumberOfProjections.unique()), 'projections with a size of', end=' ')
    for cs in Data[Data.Voxelsize == vs].CameraSize.unique():
        print(cs[0], end=' ')
    print('x', end=' ')
    for cs in Data[Data.Voxelsize == vs].CameraSize.unique():
        print(cs[1], end=' ')
    print('pixels', end=' ')
    if Data[Data.Voxelsize == vs].Wide.unique().tolist()[0]:
        print('(' + " or ".join(str(value) for value in Data[Data.Voxelsize == vs].Wide.unique()), 'projections stitched laterally)', end=' ')
    print('at every',
           str(" or ".join(str(value) for value in Data[Data.Voxelsize == vs].RotationStep.unique())) + '° over ', end='')
    if len(Data[Data.Voxelsize == vs].ThreeSixtyScan.unique()) > 1:
         print('either 180° or 360°', end=' ')
    else:
        if Data[Data.Voxelsize == vs].ThreeSixtyScan.unique()[0]:
            print('360°', end=' ')
        else:
            print('180°', end=' ')
    print('sample rotation.')
    print(80*'-')

In [ ]:
for vs in Data.Voxelsize.unique():
    print('For each sample scanned with %s μm we exposed every single projection for (on average)' % round(vs), end=' ')
    print('%s ms' % round(Data[Data.Voxelsize == vs].Exposure.mean()))
    print(80*'-')

In [ ]:
for vs in Data.Voxelsize.unique():
    print('This resulted in a average scan time of ', end='')
    print(timeformat(datetime.timedelta(seconds=Data[Data.Voxelsize == vs].Duration.mean()),
                     '{hours} hours and {minutes} minutes for each of the *stacks*'))
    print(80*'-')

In [ ]:
print('The projection images were then subsequently reconstructed into a 3D stack',
      'of images with',
      Data.Version.unique()[0][0],
      '(Version',
      Data.Version.unique()[0][1] + ', Bruker microCT, Kontich Belgium).')

In [ ]:
print('The whole process resulted in datasets with an isometric voxel size of',
      " or ".join(str(value) for value in Data.Voxelsize_rounded.unique()),
      'µm.') 